In [ ]:
import datetime
import web3

In [ ]:
rpc = "https://rpc.ankr.com/gnosis"
# rpc = "https://rpc.gnosischain.com"
w3 = web3.Web3(web3.Web3.HTTPProvider(rpc))

In [ ]:
wallet = "0x008eEB180F9Fb7190b8f2C0Ed1bEEf92Fb2Ddf32"

vault = w3.eth.contract(
    address="0xaf204776c7245bF4147c2612BF6e5972Ee483701",
    abi=[
        {
            "inputs": [{"internalType": "address", "name": "owner", "type": "address"}],
            "name": "maxWithdraw",
            "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function",
        }
    ],
)

interest = w3.eth.contract(
    address="0x670daeaF0F1a5e336090504C68179670B5059088",
    abi=[
        {
            "inputs": [],
            "name": "vaultAPY",
            "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function",
        }
    ],
)

In [ ]:
blocks = [33210412, 33210533, 33838226, 34865340, "latest"]
wallet_balance = 0
start_time = 0
total_delta_balance = 0

weighted_apr = 0
total_weight = 0
for i in range(len(blocks) - 1):
    block = blocks[i]
    next_block = blocks[i + 1]
    next_block = next_block - 1 if next_block != "latest" else next_block

    date = w3.eth.get_block(block)["timestamp"]
    next_date = w3.eth.get_block(next_block)["timestamp"]

    balance = vault.functions.maxWithdraw(wallet).call(block_identifier=block)
    next_balance = vault.functions.maxWithdraw(wallet).call(block_identifier=next_block)

    if i == 0:
        start_time = date
    if next_block == "latest":
        wallet_balance = next_balance

    delta_balance = next_balance - balance
    delta_time = next_date - date

    total_delta_balance += delta_balance

    weighted_apr += (delta_balance / balance) * 31_536_000
    total_weight += delta_time

print(f"Vault APY: {interest.functions.vaultAPY().call() / 1e16:.2f}%")
print(f"Wallet balance: ${wallet_balance / 1e18:.2f}")

delta_time = datetime.datetime.now() - datetime.datetime.fromtimestamp(start_time)

print(f"\nOverall performance:")
print(
    f"Running for {delta_time.days} days {delta_time.seconds // 3600} hours {delta_time.seconds % 3600 // 60} minutes"
)
print(f"Balance change: ${total_delta_balance / 1e18:.2f}")
print(f"APR: {weighted_apr / total_weight * 100:.2f}%")